# IQM ONE

this script will test "IQM ONE" mode.in this mode one segments is generated containing two interleaved signals: sin and cos.

when a sin signal is modulated with a cos signal, only one sideband signal relative to the LO signal

should be present.

connect channel 1 to a spectrum anlayzer at center frequency of 100MHz.

connect channel 2 to a spectrum anlayzer at center frequency of 120MHz.

connect channel 3 to a spectrum anlayzer at center frequency of 130MHz.

connect channel 4 to a spectrum anlayzer at center frequency of 140MHz.

In [1]:
def getIntepolation(i):
    switcher={
                1:'X1',
                2:'X2',
                4:'X4',
                8:'X8'
             }
    return switcher.get(i,"Invalid Interpolation")

def getMaxRate(i):
    switcher={
                'X1': 2.5e9,
                'X2': 5.0e9,
                'X4': 9.0e9,
                'X8': 9.0e9,
             }
    return switcher.get(i,"2500")

# select here the Interpolation mode : 1,2,4 or 8
Interpolation = getIntepolation(8)
maxRate = getMaxRate(Interpolation)

        
print('Interpolation = ',Interpolation)
print('maxRate = ',maxRate)

Interpolation =  X8
maxRate =  9000000000.0


In [2]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst

In [3]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

In [5]:
pid = os.getpid()
print('process id {0}'.format(pid))

# Connect to instrument
Debug = False

# Use lib_dir_path = None 
# for default location (C:\Windows\System32)
# Change it only if you know what you are doing

lib_dir_path = None
#lib_dir_path = r'D:\Projects\ProteusAwg.Nitzan\x64\Debug'


admin = TepAdmin(lib_dir_path)

slots = admin.get_slot_ids()
sid = slots[0]
assert(sid < 255)

inst = admin.open_instrument(slot_id=sid)
inst.default_paranoia_level = 2 # 0, 1 or 2

process id 20152


In [6]:
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# all IQM modes are in 16bit mode
dac_mode = 16


if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

Tabor Electronics,P2584M,000000220725,1.115.0
0, no error


## Configure Generator

In [7]:
# set Proteus to interpolation X1. this will move the module to 16bit mode if it was in 8.
inst.send_scpi_cmd(':SOUR:INT X1')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


#### Create one signal from two waves

In [8]:
# Build two AWG segment one for I and one for Q

cycleLen = 1024
numCycles = 1
segLen1 = cycleLen * numCycles

# Build waveform (cosine wave)
x1 = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen1, endpoint=False)

I = (np.cos(x1) + 1.0) * half_dac
I = np.round(I)
I = np.clip(I, 0, max_dac)
I = I.astype(data_type)

# Build waveform (sine wave)

Q = (np.sin(x1) + 1.0) * half_dac
Q = np.round(Q)
Q = np.clip(Q, 0, max_dac)
Q = Q.astype(data_type)

# build an interleaved signal for ONE mode
# the signal will be in the length of 2xsegLen

s1 = np.full((2*segLen1,1),0)
for i in range(segLen1):
    s1[0+i*2] = I[i]
    s1[1+i*2] = Q[i]
    
s1 = s1.astype(data_type)

In [9]:
# Build two AWG segment one for I and one for Q

cycleLen = 512
numCycles = 1
segLen2 = cycleLen * numCycles

# Build waveform (cosine wave)
x2 = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen2, endpoint=False)

I = (np.cos(x2) + 1.0) * half_dac
I = np.round(I)
I = np.clip(I, 0, max_dac)
I = I.astype(data_type)

# Build waveform (sine wave)


Q = (np.sin(x2) + 1.0) * half_dac
Q = np.round(Q)
Q = np.clip(Q, 0, max_dac)
Q = Q.astype(data_type)

# build an interleaved signal for ONE mode
# the signal will be in the length of 2xsegLen

s2 = np.full((2*segLen2,1),0)
for i in range(segLen2):
    s2[0+i*2] = I[i]
    s2[1+i*2] = Q[i]
    
s2 = s2.astype(data_type)

In [10]:
# Build two AWG segment one for I and one for Q

cycleLen = 256
numCycles = 1
segLen3 = cycleLen * numCycles

# Build waveform (cosine wave)
x3 = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen3, endpoint=False)

I = (np.cos(x3) + 1.0) * half_dac
I = np.round(I)
I = np.clip(I, 0, max_dac)
I = I.astype(data_type)

# Build waveform (sine wave)


Q = (np.sin(x3) + 1.0) * half_dac
Q = np.round(Q)
Q = np.clip(Q, 0, max_dac)
Q = Q.astype(data_type)

# build an interleaved signal for ONE mode
# the signal will be in the length of 2xsegLen

s3 = np.full((2*segLen3,1),0)
for i in range(segLen3):
    s3[0+i*2] = I[i]
    s3[1+i*2] = Q[i]
    
s3 = s3.astype(data_type)

In [11]:
# Build two AWG segment one for I and one for Q

cycleLen = 128
numCycles = 1
segLen4 = cycleLen * numCycles

# Build waveform (cosine wave)
x4 = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen4, endpoint=False)

I = (np.cos(x4) + 1.0) * half_dac
I = np.round(I)
I = np.clip(I, 0, max_dac)
I = I.astype(data_type)

# Build waveform (sine wave)


Q = (np.sin(x4) + 1.0) * half_dac
Q = np.round(Q)
Q = np.clip(Q, 0, max_dac)
Q = Q.astype(data_type)

# build an interleaved signal for ONE mode
# the signal will be in the length of 2xsegLen

s4 = np.full((2*segLen4,1),0)
for i in range(segLen4):
    s4[0+i*2] = I[i]
    s4[1+i*2] = Q[i]
    
s4 = s4.astype(data_type)

#### Download signals

In [12]:
# download signal to segment 1 channel 1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEF 1,' + str(2*segLen1))
inst.send_scpi_cmd(':TRAC:SEL 1')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', s1)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# download signal to segment 2 channel 2
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEF 2,' + str(2*segLen2))
inst.send_scpi_cmd(':TRAC:SEL 2')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', s2)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# download signal to segment 1 channel 3
inst.send_scpi_cmd(':INST:CHAN 3')
inst.send_scpi_cmd(':TRAC:DEF 1,' + str(2*segLen3))
inst.send_scpi_cmd(':TRAC:SEL 1')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', s3)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# download signal to segment 2 channel 4
inst.send_scpi_cmd(':INST:CHAN 4')
inst.send_scpi_cmd(':TRAC:DEF 2,' + str(2*segLen4))
inst.send_scpi_cmd(':TRAC:SEL 2')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', s4)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


0, no error
0, no error
0, no error
0, no error


#### DAC clock and output

In [13]:
# AWG Channl 1 (output) should be connected to scope input 1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 1')
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# AWG Channl 2 (output) should be connected to scope input 2
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 2')
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# AWG Channl 3 (output) should be connected to scope input 3
inst.send_scpi_cmd(':INST:CHAN 3')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 1')
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# AWG Channl 4 (output) should be connected to scope input 4
inst.send_scpi_cmd(':INST:CHAN 4')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 2')
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# chande DAC clock to 2500MHz
inst.send_scpi_cmd(':SOUR:FREQ 2.5e9')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error
0, no error
0, no error
0, no error
0, no error


#### at this point we should see a sin wave and a cos wave on the scope

#### switching to modulation mode

In [14]:
# set Proteus to interpolation X8.
inst.send_scpi_cmd(':SOUR:INT ' + str(Interpolation))
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set NCOab frequency of CH1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':SOUR:CFR 100.0e6')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set NCOab frequency of CH2
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':SOUR:CFR 120.0e6')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set NCOab frequency of CH3
inst.send_scpi_cmd(':INST:CHAN 3')
inst.send_scpi_cmd(':SOUR:CFR 130.0e6')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set NCOab frequency of CH4
inst.send_scpi_cmd(':INST:CHAN 4')
inst.send_scpi_cmd(':SOUR:CFR 140.0e6')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set modulation to ONE
inst.send_scpi_cmd(':SOUR:IQM ONE')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# chande DAC clock to 9000Hz
inst.send_scpi_cmd(':SOUR:FREQ ' + str(maxRate))
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


0, no error
0, no error
0, no error
0, no error
0, no error
0, no error
0, no error


In [15]:
inst.send_scpi_cmd(':INST:CHAN 1', paranoia_level=0)

0

## read debug registers

In [16]:
import tempfile
import webbrowser

if Debug == True :
    channb = 1
    cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(channb)
    html_str = inst.send_scpi_query(cmd, max_resp_len=200000)
    #print(html_str)
    with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
        url = 'file://' + f.name
        f.write(html_str)
    webbrowser.open(url)

## End

In [17]:
# close connection
inst.close_instrument()
admin.close_inst_admin()

0